### Compare different formulation of the Peierls and Yield stress.

In the note book, I compared the values from yield criterias and Peierls rheologies.

In [ ]:
import os, sys
import numpy as np
from shutil import rmtree, copy
from matplotlib import pyplot as plt
from matplotlib import gridspec, cm

# directory to the aspect Lab
ASPECT_LAB_DIR = os.environ['ASPECT_LAB_DIR']
RESULT_DIR = os.path.join(ASPECT_LAB_DIR, 'results')

sys.path.append(os.path.join(ASPECT_LAB_DIR))


import shilofue.Rheology as Rheology
import shilofue.ThermalModel as ThermalModel


# define global parameters
g = 9.81
rho_m = 3300.0 # mantle density
year = 365.25 * 24 * 3600.0  # s in year
Myr = 1e6 * year

#### Gerya et al 2002

This paper introduces a wet byerlee law.
The profile of yield stress predepends on a profile of the factor of the core fluid pressure ($\lambda$)

The profile of the factor of the core fluid pressure ($\lambda$) is generated from a linear relation.
The factor at the surface is assumed to be 0.4, while the factor at 10 km depth is either dry or lithostatic.

The condition for a lithostatic condition at 10 km depth is from Etheridge etal : $\lambda = 0.99$.

In [ ]:
def Gerya_2002_lambda(z, condition):
    '''
    Expression of pore fluid pressure factor lambda
    '''
    assert(condition in [0, 1])
    a = 0.4 # hydrostatic pore fluid pressure at the surface
    b = 0.0 # dry at z = 10 km, condition = 0
    if condition == 1:
        # lithostatic pore fluid pressure at z = 10 km
        b = 0.99
    _lambda = (a * (1e4 - z) + b * z) / 10000

    return _lambda

z = 5e3
_lambda_dry_5km = Gerya_2002_lambda(5e3, 0)
print("lambda at 5km assuming dry condition at 10 km: ", _lambda_dry_5km)

_lambda_wet_5km = Gerya_2002_lambda(5e3, 1)
print("lambda at 5km assuming lithostatic pore fluid pressure condition at 10 km: ", _lambda_wet_5km)

##### Relation between variables

The shear stress ($\tau$) is half the value of the differential stress ($\sigma_d$).

The differential stress ($\sigma_d$) is twice the second invariant of stress ($\sigma_{ii}$)

The effective viscosity is computed from the second invariants.

$$\tau = 1/2 \sigma_d$$
$$\sigma_d = 2 \sigma_{ii}$$
$$\eta_{eff} = \frac{\sigma_{ii}}{2 \dot{\epsilon_{ii}}}$$

So if we formulate yield stress from the shear stress, the factor of 2 should be in the deliminator.
If we formulate yield- stress from the differential stress, then there should be a factor of 4 ?

In [ ]:
zs_t2002 = np.linspace(0.0, 10e3, 1000)
Ps = zs_t2002 * rho_m * g
lambdas_0 = Gerya_2002_lambda(zs_t2002, 0)
taus_0 = Rheology.Byerlee(Ps, _lambda=lambdas_0)
lambdas_1 = Gerya_2002_lambda(zs_t2002, 1)
taus_1 = Rheology.Byerlee(Ps, _lambda=lambdas_1)

n_z = np.argmax(taus_1)
tau_1_max = taus_1[n_z]
z_1_max = zs_t2002[n_z]
print("For 1, the maximum shear stress %.2f MPa is reached at depth %.2f km." % (tau_1_max/1e6, z_1_max/1e3))

fig = plt.figure(tight_layout=True, figsize=(10, 5))
gs = gridspec.GridSpec(1, 2)

# shear stress vs depth
ax = fig.add_subplot(gs[0, 0])
ax.plot(taus_0/1e6, zs_t2002/1e3, label="Dry condition at 10 km")
ax.plot(taus_1/1e6, zs_t2002/1e3, label="Lithostatic pore fluid pressure at 10 km")
ax.set_xlabel("Shear Stress (MPa)")
ax.invert_yaxis()
ax.set_ylabel("Depth (km)")
ax.grid()
ax.legend()

# viscosity vs depth
strain_rate = 1e-15  # second invariant
etas_0_t2002 = taus_0 / 2.0 / strain_rate
etas_1_t2002 = taus_1 / 2.0 / strain_rate
ax = fig.add_subplot(gs[0, 1])
ax.semilogx(etas_0_t2002, zs_t2002/1e3, label="Dry condition at 10 km")
ax.semilogx(etas_1_t2002, zs_t2002/1e3, label="Lithostatic pore fluid pressure at 10 km")
ax.set_xlabel("Viscosity (Pa s)")
ax.invert_yaxis()
ax.set_ylabel("Depth (km)")
ax.grid()
ax.legend()
print("For 1, the maximum effective viscosity is %.2e Pa (strain rate %.2e)" % (etas_1_t2002[n_z], strain_rate))

#### Example of a coulumb yielding from Garel etal 2014

A coulumb yielding criteria  (termed byerlee law otherwise. Actually I prefer to use the term Byerlee law for the specific experimentally determined law.):

${\mu}_{y} = \frac{min(\sigma_0 + f_c P, {\tau}_{y,max})}{2\dot{\epsilon_{ii}}}$

In modelling it's usually applied in combination with a maximum yield stress. 
Usually this value only takes effect below 30 km.
One can see this by adjusting the maximum depth of the following code block.

In the block, the values of variables are taken from the paper.

In [ ]:
zs_g2014 = np.linspace(0.0, 10e3, 1000)
Ps = zs_g2014 * rho_m * g
tau_0 = 2e6
tau_m = 1000e6
friction = 0.2
taus_g2014 = Rheology.CoulumbYielding(Ps, tau_0, friction)

mask_limit = (taus_g2014 > tau_m)
taus_g2014[mask_limit] = tau_m


fig = plt.figure(tight_layout=True, figsize=(15, 5))
gs = gridspec.GridSpec(1, 3)

# shear stress vs depth
ax = fig.add_subplot(gs[0, 0])
ax.plot(taus_g2014/1e6, zs_g2014/1e3, label="Garel 2014")
ax.set_xlabel("Shear Stress (MPa)")
ax.invert_yaxis()
ax.set_ylabel("Depth (km)")
ax.grid()
ax.legend()

# viscosity vs depth
strain_rate = 1e-15  # second invariant
etas_g2014 = taus_g2014 / 2.0 / strain_rate
ax = fig.add_subplot(gs[0, 1])
ax.semilogx(etas_g2014, zs_g2014/1e3, label="Garel 2014")
ax.set_xlabel("Viscosity (Pa s)")
ax.invert_yaxis()
ax.set_ylabel("Depth (km)")
ax.grid()
ax.set_title("strain rate = %.2e" % strain_rate)

# viscosity vs depth
strain_rate = 1e-13  # second invariant
etas_g2014 = taus_g2014 / 2.0 / strain_rate
ax = fig.add_subplot(gs[0, 2])
ax.semilogx(etas_g2014, zs_g2014/1e3, label="Garel 2014")
ax.set_xlabel("Viscosity (Pa s)")
ax.invert_yaxis()
ax.set_ylabel("Depth (km)")
ax.grid()
ax.set_title("strain rate = %.2e" % strain_rate)

#### Another example of Comloumb yielding, Agrusta 2017

I show this another example just to show a variation of values people used.
The author of this paper wrote their flow law as:
$${\mu}_{byerlee} = \frac{min(\sigma_0 + f_c P, {\sigma}_{max})}{\dot{\epsilon_{ii}}}$$
Where there is missing a factor of 2.0 in the deliminator.

In [ ]:
zs_a2017 = np.linspace(0.0, 10e3, 1000)
Ps = zs_a2017 * rho_m * g
tau_0 = 20e6
tau_m = 300e6
friction = 0.2
taus_a2017 = Rheology.CoulumbYielding(Ps, tau_0, friction)

mask_limit = (taus_a2017 > tau_m)
taus_a2017[mask_limit] = tau_m

fig = plt.figure(tight_layout=True, figsize=(15, 5))
gs = gridspec.GridSpec(1, 3)

# shear stress vs depth
ax = fig.add_subplot(gs[0, 0])
ax.plot(taus_a2017/1e6, zs_a2017/1e3, label="Agrusta 2017")
ax.set_xlabel("Shear Stress (MPa)")
ax.invert_yaxis()
ax.set_ylabel("Depth (km)")
ax.grid()
ax.legend()

# viscosity vs depth
strain_rate = 1e-15  # second invariant
etas_a2017 = taus_a2017 / 2.0 / strain_rate
ax = fig.add_subplot(gs[0, 1])
ax.semilogx(etas_a2017, zs_a2017/1e3, label="Agrusta 2017")
ax.set_xlabel("Viscosity (Pa s)")
ax.invert_yaxis()
ax.set_ylabel("Depth (km)")
ax.grid()
ax.set_title("strain rate = %.2e" % strain_rate)

# viscosity vs depth
strain_rate = 1e-13  # second invariant
etas_a2017 = taus_a2017 / strain_rate
ax = fig.add_subplot(gs[0, 2])
ax.semilogx(etas_a2017, zs_a2017/1e3, label="Agrusta 2017")
ax.set_xlabel("Viscosity (Pa s)")
ax.invert_yaxis()
ax.set_ylabel("Depth (km)")
ax.grid()
ax.set_title("strain rate = %.2e" % strain_rate)

#### A third example of the Couloumb yielding: Billen 2018
This paper uses a very small value of surface cohesion of 0.1 MPa.
As we can see, this parameter is dominant in the first few kms, but have minimal impact below 10 km where the yield stress is on the order of 100 MPa.
Other paper chose to increase this value (also I chose to do that) because a small value maybe lead to strain rate localization along the surface and cause numerical issue.

In [ ]:
zs_b2018 = np.linspace(0.0, 100e3, 1000)
Ps = zs_b2018 * rho_m * g
tau_0 = 0.1e6
tau_m = 1000e6
friction = 0.6
_lambda = 0.1 # ? 0.1 or 0.9
taus_b2018 = Rheology.CoulumbYielding(Ps, tau_0, friction, _lambda)

mask_limit = (taus_b2018 > tau_m)
taus_b2018[mask_limit] = tau_m

fig = plt.figure(tight_layout=True, figsize=(15, 5))
gs = gridspec.GridSpec(1, 3)

# shear stress vs depth
ax = fig.add_subplot(gs[0, 0])
ax.plot(taus_b2018/1e6, zs_b2018/1e3, label="Billen 2018")
ax.set_xlabel("Shear Stress (MPa)")
ax.invert_yaxis()
ax.set_ylabel("Depth (km)")
ax.grid()
ax.legend()

# viscosity vs depth
strain_rate = 1e-15  # second invariant
etas_b2018 = taus_b2018 / 2.0 / strain_rate
ax = fig.add_subplot(gs[0, 1])
ax.semilogx(etas_b2018, zs_b2018/1e3, label="Billen 2018")
ax.set_xlabel("Viscosity (Pa s)")
ax.invert_yaxis()
ax.set_ylabel("Depth (km)")
ax.grid()
ax.set_title("strain rate = %.2e" % strain_rate)

# viscosity vs depth
strain_rate = 1e-13  # second invariant
etas_b2018 = taus_b2018 / 2.0/ strain_rate
ax = fig.add_subplot(gs[0, 2])
ax.semilogx(etas_b2018, zs_b2018/1e3, label="Billen 2018")
ax.set_xlabel("Viscosity (Pa s)")
ax.invert_yaxis()
ax.set_ylabel("Depth (km)")
ax.grid()
ax.set_title("strain rate = %.2e" % strain_rate)

#### The Peierls stress
#####  The stress limiter approach
This approach is mentioned in the Cizcova 2013 paper.
In this paper, the author uses a value of $\tau_y = 500 MPa$ and we can see the yield stress is mostly constrained by this variable.

The dependence on the strain rate represents a strain rate strengthening.
With 2 orders of magnitude change in the strain rate, the Peierls stress increases by 1.5 fold.

This approach is not T dependent.
In terms of the strength profile, I think the advantage is to control the value of the Peierls stress regardless of the depth and thermal gradient.

In [ ]:
tau_y = 500e6 # 500 Mpa
strain_rate_y = 1e-15
ny = 10
strain_rate = 1e-15
tau_sl_0 = Rheology.PeierlsStressLimiter(tau_y, strain_rate_y, ny, strain_rate)
eta_y_sl_0 = tau_sl_0 / strain_rate
print("At strain rate = %.2e, the yield stress is %.2e and the yield viscosity is %.2e" % (strain_rate, tau_sl_0, eta_y_sl_0))
strain_rate = 1e-14
tau_sl_1 = Rheology.PeierlsStressLimiter(tau_y, strain_rate_y, ny, strain_rate)
eta_y_sl_1 = tau_sl_1 / strain_rate
print("At strain rate = %.2e, the yield stress is %.2e and the yield viscosity is %.2e" % (strain_rate, tau_sl_1, eta_y_sl_1))
strain_rate = 1e-13
tau_sl_2 = Rheology.PeierlsStressLimiter(tau_y, strain_rate_y, ny, strain_rate)
eta_y_sl_2 = tau_sl_2 / strain_rate
print("At strain rate = %.2e, the yield stress is %.2e and the yield viscosity is %.2e" % (strain_rate, tau_sl_2, eta_y_sl_2))

#### The Mei 2010 flow law

This is a flow law of Peierls creep from the Mei 2010 paper.
This flow law might be weaker than the Evans_Goetz 1979 flow law but should be stronger than all the other ones (e.g. the Idrissi 2016).

This is also a strain rate strengthening mechanism.
By looking the values, the effect is also about 1.5 fold when the strain rate increases 2 orders.

In the shallowest 30 km, this flow law has higher value of stress than 1000 Mpa.
Thus in a strength profile, it's cut off by the brittle yielding.

At deeper depth, this will be rewriten completely by ductile flow laws, so the effective range of depth is roughly 30 - 60 km.

In [ ]:
zs_peierls = np.linspace(0.0, 100e3, 1000)
Ps = zs_peierls * rho_m * g

age = 80 * Myr
PlateModel = ThermalModel.PLATE_MODEL(100e3, 1e-6, 273.15, 1573.0)
Ts = PlateModel.T(zs_peierls, age)

creep = Rheology.GetPeierlsRheology("MK10")

strain_rate = 1e-15
taus_peierls = np.zeros(zs_peierls.size) # Mpa
for i in range(zs_peierls.size):
    taus_peierls[i] = Rheology.PeierlsCreepStress(creep, strain_rate, Ps[i], Ts[i])
eta_peierls1 = taus_peierls * 1e6 / 2.0 / strain_rate

fig = plt.figure(tight_layout=True, figsize=(15, 5))
gs = gridspec.GridSpec(1, 3)

# T vs depth
ax0 = fig.add_subplot(gs[0, 0])
ax0.plot(Ts, zs_peierls/1e3, label="Plate model with thikness of 100 km")
ax0.set_xlabel("T (k)")
ax0.invert_yaxis()
ax0.set_ylabel("Depth (km)")
ax0.grid()

# shear stress vs depth
ax1 = fig.add_subplot(gs[0, 1])
ax1.plot(taus_peierls, zs_peierls/1e3, label="Peierls %s" % strain_rate)
ax1.set_xlabel("Peierls Stress (MPa)")
ax1.invert_yaxis()
ax1.set_ylabel("Depth (km)")
ax1.grid()

# viscosity vs depth
etas_peierls = taus_peierls*1e6 / 2.0 / strain_rate
ax2 = fig.add_subplot(gs[0, 2])
ax2.semilogx(eta_peierls1, zs_peierls/1e3, label="Peierls %s" % strain_rate)
ax2.set_xlabel("Viscosity (Pa s)")
ax2.invert_yaxis()
ax2.set_ylabel("Depth (km)")
ax2.grid()

strain_rate = 1e-13
taus_peierls = np.zeros(zs_peierls.size)
for i in range(zs_peierls.size):
    taus_peierls[i] = Rheology.PeierlsCreepStress(creep, strain_rate, Ps[i], Ts[i])
eta_peierls2 = taus_peierls * 1e6 / 2.0 / strain_rate

ax1.plot(taus_peierls, zs_peierls/1e3, label="Peierls %s" % strain_rate)
ax2.semilogx(eta_peierls2, zs_peierls/1e3, label="Peierls %s" % strain_rate)

ax1.legend()

#### The Strength profile
In this block, I plot a strength profile combining brittle yielding, low-T plasticity and the dutile flow laws

In [ ]:
# pressure profile
zs_test = np.linspace(0.0, 100e3, 1000)
Ps = zs_test * rho_m * g

# thermal model
age = 80 * Myr
PlateModel = ThermalModel.PLATE_MODEL(150e3, 1e-6, 273.15, 1673.0)
Ts = PlateModel.T(zs_test, age)

# strain rate to use
strain_rate = 1e-15

# brittle yielding
tau_0 = 50e6
tau_m = 500e6
friction = 0.47
taus_test_brittle = Rheology.CoulumbYielding(Ps, tau_0, friction)
mask_limit = (taus_test_brittle > tau_m)
taus_test_brittle[mask_limit] = tau_m
eta_brittle = taus_test_brittle / 2.0 / strain_rate

# peierls creep
creep = Rheology.GetPeierlsRheology("MK10")
taus_peierls = np.zeros(zs_test.size) # Mpa
for i in range(zs_test.size):
    taus_peierls[i] = Rheology.PeierlsCreepStress(creep, strain_rate, Ps[i], Ts[i])
eta_peierls = taus_peierls * 1e6 / 2.0 / strain_rate

# diffusion & dislocation creep
# here I took the parameterization for my 2d cases
da_file = os.path.join(ASPECT_LAB_DIR, 'tests', 'integration', 'fixtures', 'test_rheology', 'depth_average.txt')
assert(os.path.isfile(da_file))
Operator = Rheology.RHEOLOGY_OPR()
Operator.ReadProfile(da_file)
rheology_aspect, viscosity_profile = Operator.MantleRheology(rheology="HK03_wet_mod", dEdiff=-40e3, dEdisl=30e3,\
dVdiff=-5.5e-6, dVdisl=2.12e-6, dAdiff_ratio=0.33333247873, dAdisl_ratio=1.040297619,\
jump_lower_mantle=15.0, Coh=300.0)
diffusion_aspect = rheology_aspect['diffusion_creep']
eta_diffusion = Rheology.CreepRheologyInAspectViscoPlastic(diffusion_aspect, strain_rate, Ps, Ts)
taus_diffusion = 2 * strain_rate * eta_diffusion
dislocation_aspect = rheology_aspect['dislocation_creep']
eta_dislocation = Rheology.CreepRheologyInAspectViscoPlastic(dislocation_aspect, strain_rate, Ps, Ts)
taus_dislocation = 2 * strain_rate * eta_dislocation

eta_dfds = 1.0 / (1.0 / eta_diffusion + 1.0 / eta_dislocation)
taus_dfds = 2 * strain_rate * eta_dfds


# get the composite rheology
eta = 1.0 / (1.0/eta_brittle + 1.0/eta_peierls + 1.0/eta_dfds)
eta1 = np.minimum(eta_peierls, eta_dfds)  # minimum
eta1 = np.minimum(eta_brittle, eta1)
eta2 = np.minimum(eta_brittle, 1.0 / (1.0 / eta_peierls + 1.0/eta_dfds)) # DP yield
eta3 = np.minimum(eta_brittle, 1.0/(1.0 / np.minimum(eta_peierls, eta_dislocation) + 1.0 / eta_diffusion)) # competing Peierls and Dislocation
eta_nopc = np.minimum(eta_brittle, eta_dfds)
taus = 2 * strain_rate * eta
taus1 = 2 * strain_rate * eta1
taus2 = 2 * strain_rate * eta2
taus3 = 2 * strain_rate * eta3
taus_nopc = 2 * strain_rate * eta_nopc

# get the minimum
taus_minimum = np.minimum(taus_test_brittle, taus_peierls*1e6)
taus_minimum = np.minimum(taus_minimum, taus_dfds)

fig = plt.figure(tight_layout=True, figsize=(10, 10))
gs = gridspec.GridSpec(2, 2)

# shear stress vs depth
ax = fig.add_subplot(gs[0, 0])
ax.plot(taus2/1e6, zs_test/1e3, label="Composite, DP yield")
ax.plot(taus_nopc/1e6, zs_test/1e3, label="Composite (no peierls)")
ax.plot(taus_test_brittle/1e6, zs_test/1e3, 'b--', label="Brittle")
ax.plot(taus_peierls, zs_test/1e3, 'c--', label="Peierls") # peierls is MPa
ax.plot(taus_dfds/1e6, zs_test/1e3, 'g--', label="Diff-Disl")
ax.set_xlabel("Shear Stress (MPa)")
ax.set_xlim([0, tau_m/1e6 * 1.1])
ax.invert_yaxis()
ax.set_ylabel("Depth (km)")
ax.grid()
ax.legend()

# viscosity vs depth
ax = fig.add_subplot(gs[0, 1])
ax.semilogx(eta2, zs_test/1e3, label="Composite, DP yield")
ax.semilogx(eta_brittle, zs_test/1e3, 'b--', label="Brittle")
ax.semilogx(eta_peierls, zs_test/1e3, 'c--', label="Peierls")
ax.semilogx(eta_dfds, zs_test/1e3, 'g--', label="Diff-Disl")
ax.set_xlabel("Viscosity (Pa s)")
ax.set_xlim([np.amin(eta)/10.0, np.amax(eta)*10.0])
ax.invert_yaxis()
ax.set_ylabel("Depth (km)")
ax.grid()
ax.set_title("strain rate = %.2e" % strain_rate)

# shear stress vs depth
ax = fig.add_subplot(gs[1, 0])
ax.plot(taus2/1e6, zs_test/1e3, label="Composite, DP yield")
ax.plot(taus/1e6, zs_test/1e3, label="Composite, SL yield")
ax.plot(taus_nopc/1e6, zs_test/1e3, label="Composite (no peierls)")
ax.plot(taus1/1e6, zs_test/1e3, label="Minimization")
ax.plot(taus3/1e6, zs_test/1e3, label="Competing Peierls and dislocation")
ax.set_xlabel("Shear Stress (MPa)")
ax.set_xlim([0, tau_m/1e6 * 1.1])
ax.invert_yaxis()
ax.set_ylabel("Depth (km)")
ax.grid()
ax.legend()

fig_path = os.path.join(RESULT_DIR, "Peierls_Yielding_Compare_Strength_Profile_%.2e.pdf" % strain_rate)
fig.savefig(fig_path)
print("save figure: %s" % fig_path)